<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">Classification with Python</font></h1>

In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset

This dataset is about past loans. The __Loan_train.csv__ data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |

Lets download the dataset

In [2]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

/usr/local/bin/bash: line 1: wget: command not found


### Load Data From CSV File  

In [3]:
df = pd.read_csv('loan_train.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'loan_train.csv'

In [ ]:
df.shape

### Convert to date time object 

In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

# Data visualization and pre-processing



Let’s see how many of each class is in our data set 

In [ ]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection 


Lets plot some columns to underestand data better:

In [ ]:
# notice: installing seaborn might takes a few minutes
# !conda install -c conda-forge seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction

### Lets look at the day of the week people get the loan 

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values

Lets look at gender:

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Lets convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding  
#### How about education?

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Feature befor One Hot Encoding

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame 

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()


### Feature selection

Lets defind feature sets, X:

In [ ]:
X = Feature
X[0:5]

In [ ]:
X.shape

What are our lables?

In [ ]:
y = df['loan_status'].values
y[0:5]

## Normalize Data 

Data Standardization give data zero mean and unit variance (technically should be done after train test split )

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification 

Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:
- K Nearest Neighbor(KNN)
- Decision Tree
- Support Vector Machine
- Logistic Regression



__ Notice:__ 
- You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
- You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
- You should include the code of the algorithm in the following cells.

In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
X.shape

# K Nearest Neighbor(KNN)
Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the __loan_test.csv__ for finding the best k, however, you can split your train_loan.csv into train and test to find the best __k__.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

def KNN_cross_val_score_Optimizer(X, y, k):
    neigh = KNeighborsClassifier(n_neighbors = k)
    return np.asarray(cross_val_score(neigh, X, y, cv=5))

In [ ]:
K_MAX = 20
KNN_scores = np.zeros((K_MAX,5))
for k in range(1, K_MAX+1):
    KNN_score = KNN_cross_val_score_Optimizer(X, y, k)
    KNN_scores[k-1,:] = KNN_score
    
score_avg = KNN_scores.mean(axis=1)
score_std = KNN_scores.std(axis=1)

In [ ]:
plt.plot(range(1,K_MAX+1),score_avg,'g')
plt.fill_between(range(1,K_MAX+1), score_avg - 1*score_std, score_avg + 1*score_std, alpha=0.10)
plt.fill_between(range(1,K_MAX+1), score_avg - 3*score_std, score_avg + 3*score_std, alpha=0.10, color="green")
plt.legend(('Accuracy ', '+/- 1xstd','+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Neighbors (K)')
plt.xticks([1, 3, 5, 7, 9, 11, 13, 15, 17, 19])
plt.title("Best accuracy = " + str(score_avg.max().round(3)) + " with k = " + str(score_avg.argmax()+1))
plt.tight_layout()
plt.show()

In [ ]:
KNN_model = KNeighborsClassifier(n_neighbors=7)
KNN_model.fit(X, y)
train_data_score = KNN_model.score(X, y)
print('Train data score:', train_data_score)

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

def tree_cross_val_score_optimizer(X, y, max_depth):
    tree_model = DecisionTreeClassifier(criterion="entropy", max_depth=max_depth)
    return np.asarray(cross_val_score(tree_model, X, y, cv=5))

In [ ]:
K_MAX = 20
tree_scores = np.zeros((K_MAX-1,5))
for k in range(2, K_MAX+1):
    tree_score = tree_cross_val_score_optimizer(X, y, k)
    tree_scores[k-2,:] = tree_score
    
score_avg = tree_scores.mean(axis=1)
score_std = tree_scores.std(axis=1)

In [ ]:
plt.plot(range(2,K_MAX+1),score_avg,'g')
plt.fill_between(range(2,K_MAX+1), score_avg - 1*score_std, score_avg + 1*score_std, alpha=0.10)
plt.fill_between(range(2,K_MAX+1), score_avg - 3*score_std, score_avg + 3*score_std, alpha=0.10, color="green")
plt.legend(('Accuracy ', '+/- 1xstd','+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Depth of the tree')
plt.xticks([1, 3, 5, 7, 9, 11, 13, 15, 17, 19])
plt.title("Best accuracy = " + str(score_avg.max().round(3)) + " with depth = " + str(score_avg.argmax()+2))
plt.tight_layout()
plt.show()

In [ ]:
tree_model = DecisionTreeClassifier(criterion="entropy", max_depth=10)
tree_model.fit(X, y)
train_data_score = tree_model.score(X, y)
print('Train data score:', train_data_score)

# Support Vector Machine

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score

def svm_cross_val_score_optimizer(X, y, kernel):
    svm_model = svm.SVC(kernel=kernel)
    return np.asarray(cross_val_score(svm_model, X, y, cv=5))

In [ ]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
svm_scores = np.zeros((len(kernels),5))
k = 0
for kernel in kernels:
    print(kernel)
    svm_score = svm_cross_val_score_optimizer(X, y, kernel)
    svm_scores[k,:] = svm_score
    k += 1
    
score_avg = svm_scores.mean(axis=1)
score_std = svm_scores.std(axis=1)

In [ ]:
plt.plot(range(0,len(kernels)),score_avg,'g')
plt.fill_between(range(0,len(kernels)), score_avg - 1*score_std, score_avg + 1*score_std, alpha=0.10)
plt.fill_between(range(0,len(kernels)), score_avg - 3*score_std, score_avg + 3*score_std, alpha=0.10, color="green")
plt.legend(('Accuracy ', '+/- 1xstd','+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('SVM Kernel')
plt.xticks(range(0,len(kernels)), kernels)

plt.title("Best accuracy = " + str(score_avg.max().round(3)) + " with kernel = " + kernels[score_avg.argmax()])
plt.tight_layout()
plt.show()

In [ ]:
svm_model = svm.SVC(kernel='poly')
svm_model.fit(X, y)
train_data_score = svm_model.score(X, y)
print('Train data score:', train_data_score)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV

def LR_cross_val_score_optimizer(X, y, parameters):
    LR_model = LogisticRegression()
    LR_grid_search = GridSearchCV(LR_model, parameters)
    LR_grid_search.fit(X, y)
    return LR_grid_search

In [ ]:
params = {'C': [1],
          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
a = LR_cross_val_score_optimizer(X, y, params)

In [ ]:
import pandas as pd

pd.DataFrame.from_dict(a.cv_results_)

In [ ]:
LR_model = LogisticRegression(C=1, solver='sag')
LR_model.fit(X, y)
train_data_score = LR_model.score(X, y)
print('Train data score:', train_data_score)

# Model Evaluation using Test set

In [ ]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report

First, download and load the test set:

In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation 

In [ ]:
features = ['Principal','terms','age','Gender','weekend']
test_df = pd.read_csv('loan_test.csv')
test_df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1], inplace=True)
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df.head()

In [ ]:
X_test = test_df[features]
X_test = pd.concat([X_test, pd.get_dummies(test_df['education'])], axis=1)
X_test.drop(['Master or Above'], axis = 1,inplace=True)
X_test.head()

In [ ]:
X_test = preprocessing.StandardScaler().fit(X_test).transform(X_test)
X_test.shape

In [ ]:
y_test = test_df['loan_status'].values
y_test[0:5]

# Report
You should be able to report the accuracy of the built model using different evaluation metrics:

In [ ]:
# KNN test score
y_hat = KNN_model.predict(X_test)

KNN_jaccard = jaccard_score(y_test, y_hat, pos_label='PAIDOFF')
print('KNN Jaccard-score:', KNN_jaccard)
KNN_f1 = f1_score(y_test, y_hat, pos_label='PAIDOFF')
print('KNN F1-score:', KNN_f1)

print(classification_report(y_test, y_hat))

In [ ]:
# Decision tree score
y_hat = tree_model.predict(X_test)

tree_jaccard = jaccard_score(y_test, y_hat, pos_label='PAIDOFF')
print('Decision tree Jaccard-score:', tree_jaccard)
tree_f1 = f1_score(y_test, y_hat, pos_label='PAIDOFF')
print('Decision tree F1-score:', tree_f1)

print(classification_report(y_test, y_hat))

In [ ]:
# SVM score
y_hat = svm_model.predict(X_test)

svm_jaccard = jaccard_score(y_test, y_hat, pos_label='PAIDOFF')
print('SVM Jaccard-score:', svm_jaccard)
svm_f1 = f1_score(y_test, y_hat, pos_label='PAIDOFF')
print('SVM F1-score:', svm_f1)

print(classification_report(y_test, y_hat))

In [ ]:
# Logistic regression score
y_hat = LR_model.predict(X_test)
y_prob = LR_model.predict_proba(X_test)

LR_jaccard = jaccard_score(y_test, y_hat, pos_label='PAIDOFF')
print('Logistic regression Jaccard-score:', LR_jaccard)
LR_f1 = f1_score(y_test, y_hat, pos_label='PAIDOFF')
print('Logistic regression F1-score:', LR_f1)
LR_logloss = log_loss(y_test, y_prob)
print('Logistic regression LogLoss:', LR_logloss)
print(classification_report(y_test, y_hat))

| Algorithm          | Jaccard | F1-score | LogLoss |
|--------------------|---------|----------|---------|
| KNN                | 0.65    | 0.67     | NA      |
| Decision Tree      | 0.54    | 0.59     | NA      |
| SVM                | 0.74    | 0.74     | NA      |
| LogisticRegression | 0.74    | 0.74     | 0.56    |

<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p>